In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)


In [125]:
# import package

# model
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch import optim
from torch.optim.lr_scheduler import StepLR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau

# dataset and transformation
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import models
import torch.nn as nn
import torch.nn.functional as F
import torchvision


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, LSTM, Dropout, ConvLSTM2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback
from keras.regularizers import l2
# display images
from torchvision import utils
import matplotlib.pyplot as plt
%matplotlib inline


# utils
import numpy as np
from torchsummary import summary
import time
import copy
import pandas as pd

import os
from warnings import filterwarnings
filterwarnings('ignore')

import torch
from torch import nn, optim
from torch.backends import cudnn
cudnn.benchmark = True
from randaugment import RandAugment, ImageNetPolicy

import seaborn as sns
plt.rcParams['font.family'] = 'Malgun Gothic'

import tensorflow as tf

In [3]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

cuda:0


In [4]:
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None )

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [5]:
train =pd.read_csv("../data/train_preprocessing.csv")
submission = pd.read_csv("../data/sample_submission.csv")

In [6]:
train.drop('일교차',inplace=True, axis = 1)

In [7]:
train

,일시,최고기온,최저기온,강수량,평균습도,평균풍속,일조합,일사합,일조율,평균기온,년,월,일
0,1960-01-01,2.2,-5.2,-0.730379,68.3,1.7,6.7,12.562132,55.338502,-1.6,1960,1,1
1,1960-01-02,1.2,-5.6,0.400000,87.7,1.3,0.0,4.541569,2.040280,-1.9,1960,1,2
2,1960-01-03,8.7,-2.1,0.000000,81.3,3.0,0.0,5.738938,2.040280,4.0,1960,1,3
3,1960-01-04,10.8,1.2,0.000000,79.7,4.4,2.6,8.422487,22.723172,7.5,1960,1,4
4,1960-01-05,1.3,-8.2,0.705818,44.0,5.1,8.2,14.946189,67.270939,-4.6,1960,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23006,2022-12-27,3.3,-7.3,0.809723,69.8,1.8,8.8,10.250000,91.700000,-2.6,2022,12,27
23007,2022-12-28,0.1,-6.0,0.100000,58.1,2.5,8.7,10.860000,90.600000,-3.3,2022,12,28
23008,2022-12-29,2.1,-7.8,0.000000,56.3,1.7,9.0,10.880000,93.800000,-2.9,2022,12,29
23009,2022-12-30,2.3,-4.4,0.000000,65.6,1.9,7.9,10.840000,82.300000,-1.8,2022,12,30


In [8]:
train.loc[train['강수량'] < 0 , '강수량'] = 0 

In [9]:
train

,일시,최고기온,최저기온,강수량,평균습도,평균풍속,일조합,일사합,일조율,평균기온,년,월,일
0,1960-01-01,2.2,-5.2,0.000000,68.3,1.7,6.7,12.562132,55.338502,-1.6,1960,1,1
1,1960-01-02,1.2,-5.6,0.400000,87.7,1.3,0.0,4.541569,2.040280,-1.9,1960,1,2
2,1960-01-03,8.7,-2.1,0.000000,81.3,3.0,0.0,5.738938,2.040280,4.0,1960,1,3
3,1960-01-04,10.8,1.2,0.000000,79.7,4.4,2.6,8.422487,22.723172,7.5,1960,1,4
4,1960-01-05,1.3,-8.2,0.705818,44.0,5.1,8.2,14.946189,67.270939,-4.6,1960,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23006,2022-12-27,3.3,-7.3,0.809723,69.8,1.8,8.8,10.250000,91.700000,-2.6,2022,12,27
23007,2022-12-28,0.1,-6.0,0.100000,58.1,2.5,8.7,10.860000,90.600000,-3.3,2022,12,28
23008,2022-12-29,2.1,-7.8,0.000000,56.3,1.7,9.0,10.880000,93.800000,-2.9,2022,12,29
23009,2022-12-30,2.3,-4.4,0.000000,65.6,1.9,7.9,10.840000,82.300000,-1.8,2022,12,30


In [10]:
# '월' 컬럼에 대한 순환 인코딩
train['월_sin'] = np.sin(2 * np.pi * train['월'] / 12)
train['월_cos'] = np.cos(2 * np.pi * train['월'] / 12)

# '일' 컬럼에 대한 순환 인코딩
train['일_sin'] = np.sin(2 * np.pi * train['일'] / 31)
train['일_cos'] = np.cos(2 * np.pi * train['일'] / 31)

In [11]:
train.drop(['월','일'], axis = 1 , inplace= True)

In [12]:
train

,일시,최고기온,최저기온,강수량,평균습도,평균풍속,일조합,일사합,일조율,평균기온,년,월_sin,월_cos,일_sin,일_cos
0,1960-01-01,2.2,-5.2,0.000000,68.3,1.7,6.7,12.562132,55.338502,-1.6,1960,5.000000e-01,0.866025,2.012985e-01,0.979530
1,1960-01-02,1.2,-5.6,0.400000,87.7,1.3,0.0,4.541569,2.040280,-1.9,1960,5.000000e-01,0.866025,3.943559e-01,0.918958
2,1960-01-03,8.7,-2.1,0.000000,81.3,3.0,0.0,5.738938,2.040280,4.0,1960,5.000000e-01,0.866025,5.712682e-01,0.820763
3,1960-01-04,10.8,1.2,0.000000,79.7,4.4,2.6,8.422487,22.723172,7.5,1960,5.000000e-01,0.866025,7.247928e-01,0.688967
4,1960-01-05,1.3,-8.2,0.705818,44.0,5.1,8.2,14.946189,67.270939,-4.6,1960,5.000000e-01,0.866025,8.486443e-01,0.528964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23006,2022-12-27,3.3,-7.3,0.809723,69.8,1.8,8.8,10.250000,91.700000,-2.6,2022,-2.449294e-16,1.000000,-7.247928e-01,0.688967
23007,2022-12-28,0.1,-6.0,0.100000,58.1,2.5,8.7,10.860000,90.600000,-3.3,2022,-2.449294e-16,1.000000,-5.712682e-01,0.820763
23008,2022-12-29,2.1,-7.8,0.000000,56.3,1.7,9.0,10.880000,93.800000,-2.9,2022,-2.449294e-16,1.000000,-3.943559e-01,0.918958
23009,2022-12-30,2.3,-4.4,0.000000,65.6,1.9,7.9,10.840000,82.300000,-1.8,2022,-2.449294e-16,1.000000,-2.012985e-01,0.979530


In [13]:
print(train['평균기온'].max(),train['평균기온'].min())

33.7 -16.4


In [14]:
# Min-Max Scaler 초기화
scaler = MinMaxScaler()

# Min-Max Scaling을 적용할 컬럼들 선택
columns_to_scale = ['최고기온', '최저기온', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율', '평균기온', '년']

# Scaling 적용
train[columns_to_scale] = scaler.fit_transform(train[columns_to_scale])

In [15]:
train.drop('일시',axis = 1 , inplace = True)

In [16]:
train

,최고기온,최저기온,강수량,평균습도,평균풍속,일조합,일사합,일조율,평균기온,년,월_sin,월_cos,일_sin,일_cos
0,0.296992,0.297030,0.000000,0.615385,0.207792,0.489051,0.375213,0.554015,0.295409,0.0,5.000000e-01,0.866025,2.012985e-01,0.979530
1,0.278195,0.289109,0.001202,0.852259,0.155844,0.000000,0.135650,0.020426,0.289421,0.0,5.000000e-01,0.866025,3.943559e-01,0.918958
2,0.419173,0.358416,0.000000,0.774115,0.376623,0.000000,0.171414,0.020426,0.407186,0.0,5.000000e-01,0.866025,5.712682e-01,0.820763
3,0.458647,0.423762,0.000000,0.754579,0.558442,0.189781,0.251568,0.227490,0.477046,0.0,5.000000e-01,0.866025,7.247928e-01,0.688967
4,0.280075,0.237624,0.002121,0.318681,0.649351,0.598540,0.446421,0.673475,0.235529,0.0,5.000000e-01,0.866025,8.486443e-01,0.528964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23006,0.317669,0.255446,0.002433,0.633700,0.220779,0.642336,0.306153,0.918044,0.275449,1.0,-2.449294e-16,1.000000,-7.247928e-01,0.688967
23007,0.257519,0.281188,0.000300,0.490842,0.311688,0.635036,0.324373,0.907032,0.261477,1.0,-2.449294e-16,1.000000,-5.712682e-01,0.820763
23008,0.295113,0.245545,0.000000,0.468864,0.207792,0.656934,0.324970,0.939068,0.269461,1.0,-2.449294e-16,1.000000,-3.943559e-01,0.918958
23009,0.298872,0.312871,0.000000,0.582418,0.233766,0.576642,0.323775,0.823937,0.291417,1.0,-2.449294e-16,1.000000,-2.012985e-01,0.979530


In [102]:
window_size = 358*4
prediction_length = 358
X_train = []
y_train = []

# 윈도우에 따른 데이터 분할
for i in range(window_size, len(train)-357):
    X_train.append(train.iloc[i-window_size:i])
    y_train.append(train['평균기온'].iloc[i:i+358].values)  # 358일치 평균기온을 배열로 추가

In [103]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [104]:
X_train.shape

(21222, 1432, 14)

In [105]:
y_train.shape

(21222, 358)

In [106]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=3, min_lr=0.00001)

class LearningRateTracker(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        lr = self.model.optimizer.lr
        # 학습률을 실제 값으로 변환하고 float 타입으로 캐스팅
        current_lr = float(K.eval(lr))
        # 반올림
        rounded_lr = round(current_lr, 6)
        print(f'에포크 {epoch + 1}: 현재 학습률 = {rounded_lr}')

# 콜백 인스턴스 생성
lr_tracker = LearningRateTracker()

IndexError: tuple index out of range

In [123]:
# # 모델 구축
# model = Sequential()
# model.add(GRU(units=64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
# model.add(GRU(units=64 , return_sequences=True))
# model.add(GRU(units=64 )) 
# model.add(Dense(units=358 ))
# 
# # 모델 컴파일
# model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
# 
# # 모델 요약 출력
# model.summary()

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_54 (GRU)                 (None, 1432, 64)          15360     
_________________________________________________________________
gru_55 (GRU)                 (None, 1432, 64)          24960     
_________________________________________________________________
gru_56 (GRU)                 (None, 64)                24960     
_________________________________________________________________
dense_25 (Dense)             (None, 358)               23270     
Total params: 88,550
Trainable params: 88,550
Non-trainable params: 0
_________________________________________________________________


In [124]:
# 모델 훈련
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.1, verbose=1, callbacks=[reduce_lr, early_stopping, lr_tracker])

Epoch 1/50
에포크 1: 현재 학습률 = 0.001
299/299 [==============================] - 37s 110ms/step - loss: 0.0235 - val_loss: 0.0050
Epoch 2/50
에포크 2: 현재 학습률 = 0.001
299/299 [==============================] - 32s 107ms/step - loss: 0.0045 - val_loss: 0.0048
Epoch 3/50
에포크 3: 현재 학습률 = 0.001
299/299 [==============================] - 32s 107ms/step - loss: 0.0043 - val_loss: 0.0045
Epoch 4/50
에포크 4: 현재 학습률 = 0.001
299/299 [==============================] - 31s 104ms/step - loss: 0.0042 - val_loss: 0.0044
Epoch 5/50
에포크 5: 현재 학습률 = 0.001
299/299 [==============================] - 31s 104ms/step - loss: 0.0042 - val_loss: 0.0045
Epoch 6/50
에포크 6: 현재 학습률 = 0.001
299/299 [==============================] - 31s 104ms/step - loss: 0.0042 - val_loss: 0.0045
Epoch 7/50
에포크 7: 현재 학습률 = 0.0005
299/299 [==============================] - 31s 103ms/step - loss: 0.0041 - val_loss: 0.0044
Epoch 8/50
에포크 8: 현재 학습률 = 0.0005
299/299 [==============================] - 31s 104ms/step - loss: 0.0041 - val_loss: 0.004

In [109]:
# 데이터 타입과 차원 확인
print(type(X_train), X_train.shape)
print(type(y_train), y_train.shape)

<class 'numpy.ndarray'> (21222, 1432, 14)
<class 'numpy.ndarray'> (21222, 358)


In [110]:
predict_data = X_train[-1].reshape(1, -1, 14)

In [111]:
pred = model.predict(predict_data)

In [112]:
pred

array([[0.30355653, 0.29358548, 0.29948848, 0.29686874, 0.2913569 ,
        0.29516962, 0.29452828, 0.2923019 , 0.2937223 , 0.29421532,
        0.28307536, 0.28812316, 0.29130435, 0.29204068, 0.2852926 ,
        0.28917354, 0.29682398, 0.29413262, 0.2941746 , 0.29619047,
        0.29583105, 0.28877935, 0.29574317, 0.29521072, 0.29445305,
        0.2994146 , 0.29957998, 0.29942498, 0.30264974, 0.31062442,
        0.30099258, 0.3096087 , 0.3163325 , 0.32053995, 0.32160506,
        0.3246101 , 0.32513916, 0.32220507, 0.32836488, 0.33473048,
        0.33895046, 0.34580258, 0.33722085, 0.35015023, 0.34613577,
        0.34534347, 0.35088885, 0.35677677, 0.36120754, 0.37384024,
        0.37344962, 0.3719427 , 0.37430704, 0.38089737, 0.38606527,
        0.3892528 , 0.39072642, 0.39109126, 0.405672  , 0.40120518,
        0.4094008 , 0.4165422 , 0.41208085, 0.41657192, 0.42973262,
        0.43102652, 0.43435946, 0.4445545 , 0.44220015, 0.4484799 ,
        0.4471367 , 0.45039168, 0.45854214, 0.46

In [113]:
pred.shape

(1, 358)

In [114]:
# MinMaxScaler의 역변환을 수행하는 함수 정의
def inverse_minmax_scaling(scaled_data, min_val, max_val):
    return (scaled_data * (max_val - min_val)) + min_val

# 최소값과 최대값 정의
min_val = -16.4
max_val = 33.7

# 역변환 수행
pred = inverse_minmax_scaling(pred, min_val, max_val)

In [115]:
pred

array([[-1.1918182 , -1.6913671 , -1.395627  , -1.5268764 , -1.8030195 ,
        -1.6120024 , -1.6441336 , -1.7556753 , -1.6845131 , -1.6598129 ,
        -2.217924  , -1.9650297 , -1.8056517 , -1.7687626 , -2.106841  ,
        -1.912406  , -1.5291185 , -1.6639557 , -1.6618519 , -1.5608578 ,
        -1.5788641 , -1.9321547 , -1.5832672 , -1.6099434 , -1.6479025 ,
        -1.3993282 , -1.3910427 , -1.3988085 , -1.2372484 , -0.83771706,
        -1.3202715 , -0.88860416, -0.55174255, -0.3409481 , -0.2875862 ,
        -0.13703346, -0.11052704, -0.2575264 ,  0.0510807 ,  0.36999702,
         0.5814171 ,  0.9247093 ,  0.49476433,  1.1425266 ,  0.94140244,
         0.9017086 ,  1.1795311 ,  1.4745159 ,  1.696497  ,  2.3293953 ,
         2.309826  ,  2.2343292 ,  2.3527832 ,  2.6829586 ,  2.9418697 ,
         3.1015663 ,  3.175394  ,  3.1936722 ,  3.9241676 ,  3.7003803 ,
         4.110979  ,  4.4687634 ,  4.2452507 ,  4.470253  ,  5.1296043 ,
         5.1944275 ,  5.361408  ,  5.872181  ,  5.7

In [116]:
pred.shape

(1, 358)

In [117]:
pred = pred[-1].reshape(-1,)

In [118]:
pred

array([-1.1918182 , -1.6913671 , -1.395627  , -1.5268764 , -1.8030195 ,
       -1.6120024 , -1.6441336 , -1.7556753 , -1.6845131 , -1.6598129 ,
       -2.217924  , -1.9650297 , -1.8056517 , -1.7687626 , -2.106841  ,
       -1.912406  , -1.5291185 , -1.6639557 , -1.6618519 , -1.5608578 ,
       -1.5788641 , -1.9321547 , -1.5832672 , -1.6099434 , -1.6479025 ,
       -1.3993282 , -1.3910427 , -1.3988085 , -1.2372484 , -0.83771706,
       -1.3202715 , -0.88860416, -0.55174255, -0.3409481 , -0.2875862 ,
       -0.13703346, -0.11052704, -0.2575264 ,  0.0510807 ,  0.36999702,
        0.5814171 ,  0.9247093 ,  0.49476433,  1.1425266 ,  0.94140244,
        0.9017086 ,  1.1795311 ,  1.4745159 ,  1.696497  ,  2.3293953 ,
        2.309826  ,  2.2343292 ,  2.3527832 ,  2.6829586 ,  2.9418697 ,
        3.1015663 ,  3.175394  ,  3.1936722 ,  3.9241676 ,  3.7003803 ,
        4.110979  ,  4.4687634 ,  4.2452507 ,  4.470253  ,  5.1296043 ,
        5.1944275 ,  5.361408  ,  5.872181  ,  5.7542267 ,  6.06

In [119]:
sub = pd.read_csv('../data/sample_submission.csv')
sub['평균기온'] = pred

sub.to_csv('../submission/GRU_5.csv', index=False)